## Сверточные нейронные сети для анализа текста

Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [57]:
# Загрузка данных Google Colab

# from google.colab import files
# files.upload()

In [58]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [59]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
# Загрузка данных

import pandas as pd

df = pd.read_excel('/gdrive/MyDrive/Colab Notebooks/отзывы за лето.xls')
df.sample(15)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [63]:
df.Content = df.Content.astype(str)

In [64]:
# Предобработка текста

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [65]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20659 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [67]:
# Разделение выборки на тренировочную, тестовую и валидационную

from sklearn.model_selection import train_test_split

train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=17)

test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=17)

In [68]:
# Создание корпуса

train_corpus = " ".join(train_df['Content'])
train_corpus = train_corpus.lower()

In [69]:
train_corpus

'полнеприличный приложение андроид около немочь перенести приложение сд карта сделать перенос полезный удобный вещьнезнаюкак жить банкинг телефон удобно управлять свой счёт дача оплачивать дистанционно ненадо терять отдых отлично отличный приложение ребята удобный странный показаться сторона сбербанк делать код вход приложение 5 знакоми не4 сбербанк 4 кодный защита порадовать заходить писать добрый утро незнать задать вопрос регистрация вводить номер карта писать соединение интернетомчто пойти нетак пробовать сканировать непроисходить приложение нормально работать отлично 1 минус грузиться вячеслав мненравиться удобно красавец чтоть глючить последний обновление телефон постоянно перезагружаться перезагрузка остановить включение телефон удаление приложение работать нормально хороший приложение последний обновление приложение открываться постоянно проверять антивирус прошивка смысл антивирус порядок вечно находить какой ерунда приходиться сносить устанавливать инициализация антивирус при

In [70]:
len(train_corpus)

613233

In [71]:
# Токенизация корпуса

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
# Настраиваемые параметры сети

max_words = 3000
max_len = 120
num_classes = 6

In [73]:
# Настраиваемые параметры обучения

epochs = 10
batch_size = 512
print_batch_n = 100

In [74]:
# Сортировка и отбор самых частотных токенов

from nltk.probability import FreqDist

tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [75]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'удобный',
 'работать',
 'отлично',
 'хороший',
 'отличный',
 'супер',
 'обновление',
 'телефон']

In [76]:
dist[tokens_filtered_top[0]], dist[tokens_filtered_top[9]]

(4478, 549)

In [77]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [78]:
vocabulary['приложение']

1

In [79]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [80]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_df['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_df['Content']], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in val_df['Content']], dtype=np.int32)

In [81]:
x_train.shape, x_train[0]

((14461, 120),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    1,  139,   41,   14, 2460,    1, 2461,   21,   26],
       dtype=int32))

In [82]:
train_df.iloc[0].Content

'полнеприличный приложение андроид около немочь перенести приложение сд карта сделать перенос'

In [97]:
list(vocabulary.keys())[list(vocabulary.values()).index(2460)]


'перенести'

In [98]:
# Импорт библиотек для работы нейронной сети

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

In [99]:
y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes)

In [100]:
# Сеть

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [108]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [109]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 128)          384000    
                                                                 
 conv1d (Conv1D)             (None, 118, 128)          49280     
                                                                 
 activation (Activation)     (None, 118, 128)          0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                        

In [111]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
26/26 [==============================] - 12s 444ms/step - loss: 0.4946 - accuracy: 0.8326 - val_loss: 0.7349 - val_accuracy: 0.7567
Epoch 2/10
26/26 [==============================] - 12s 459ms/step - loss: 0.4665 - accuracy: 0.8412 - val_loss: 0.7508 - val_accuracy: 0.7554


In [112]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 97ms/step - loss: 0.6533 - accuracy: 0.7809


Test score: 0.6532565951347351
Test accuracy: 0.7808970808982849


In [113]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 1s 98ms/step


### Embedding слой с предобученными векторами

In [114]:
# Загрузка предобученной модели

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2022-06-16 07:33:21--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  26.1MB/s    in 14s     

2022-06-16 07:33:35 (24.3 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]



In [115]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Content'])
vocab_size = len(tokenizer.word_index) + 1

In [116]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [125]:
embeddings_index['приложение'], embeddings_index['приложение'].shape

(array([ 1.58915797e-03,  2.31340043e-02, -9.28171352e-02, -6.93971291e-02,
        -7.57938027e-02,  2.95946170e-02,  4.13068496e-02, -3.43898125e-02,
         4.79290485e-02, -3.81807536e-02,  3.19607295e-02, -1.18584558e-02,
         2.05491409e-02,  1.48728220e-02,  1.39552318e-02, -2.19477952e-04,
        -1.24828011e-01,  1.55550446e-02, -4.12986390e-02,  2.22136416e-02,
        -2.56619863e-02, -5.80532923e-02, -1.85665442e-03,  4.64064851e-02,
         5.53247295e-02, -3.67254764e-02,  1.97346032e-01,  7.42573366e-02,
        -1.11277558e-01, -7.83833414e-02,  1.38044879e-02, -9.10650492e-02,
         4.04436626e-02, -5.19908592e-02,  4.58941329e-03,  1.78939551e-02,
         8.01691785e-02, -2.91911755e-02,  1.72394201e-01, -3.14709470e-02,
         1.33355454e-01,  2.13857479e-02, -2.29125959e-03, -9.65981781e-02,
        -7.44792894e-02, -2.55136248e-02,  6.71730563e-02,  7.65780360e-02,
         3.05248369e-02, -2.25017890e-02,  2.74218246e-02,  2.43360978e-02,
         1.3

In [162]:
tokenizer.word_index.items()

dict_items([('приложение', 1), ('удобно', 2), ('удобный', 3), ('работать', 4), ('отлично', 5), ('хороший', 6), ('отличный', 7), ('супер', 8), ('обновление', 9), ('телефон', 10), ('быстро', 11), ('пароль', 12), ('антивирус', 13), ('немочь', 14), ('нравиться', 15), ('банк', 16), ('пользоваться', 17), ('сбербанк', 18), ('вход', 19), ('раз', 20), ('карта', 21), ('устраивать', 22), ('прошивка', 23), ('проблема', 24), ('рута', 25), ('сделать', 26), ('счёт', 27), ('деньга', 28), ('писать', 29), ('программа', 30), ('перевод', 31), ('мненравиться', 32), ('ошибка', 33), ('разработчик', 34), ('норма', 35), ('приходиться', 36), ('вводить', 37), ('платёж', 38), ('постоянно', 39), ('нормально', 40), ('около', 41), ('смс', 42), ('довольный', 43), ('свой', 44), ('неработать', 45), ('код', 46), ('функция', 47), ('понятно', 48), ('шаблон', 49), ('зайти', 50), ('исправить', 51), ('вылетать', 52), ('последний', 53), ('возможность', 54), ('стать', 55), ('право', 56), ('операция', 57), ('мобильный', 58), ('

In [160]:
# Матрица эмбеддингов

embedding_dim = 300
inx_emb_tokens = []
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        inx_emb_tokens.append(word)
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4886 words (4993 misses)


In [161]:
embedding_matrix.shape

(9880, 300)

In [140]:
inx_emb_tokens[:10]

['приложение',
 'удобно',
 'удобный',
 'работать',
 'отлично',
 'хороший',
 'отличный',
 'супер',
 'обновление',
 'телефон']

In [146]:
print(embedding_matrix[0] == embedding_matrix[inx_emb_tokens.index(inx_emb_tokens[0])])


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [147]:
# Сеть

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [148]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [149]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 120, 300)          2964000   
                                                                 
 conv1d_3 (Conv1D)           (None, 118, 128)          115328    
                                                                 
 activation_9 (Activation)   (None, 118, 128)          0         
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 activation_10 (Activation)  (None, 10)                0         
                                                      

In [150]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
26/26 [==============================] - 20s 737ms/step - loss: 1.5912 - accuracy: 0.1790 - val_loss: 1.4470 - val_accuracy: 0.4485
Epoch 2/10
26/26 [==============================] - 18s 690ms/step - loss: 1.2454 - accuracy: 0.6913 - val_loss: 1.1281 - val_accuracy: 0.6814


In [151]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 2s 291ms/step - loss: 1.0644 - accuracy: 0.7102


Test score: 1.064357042312622
Test accuracy: 0.7102290987968445
